In [1]:
%ls

Datasets/                                 nltk_data/
GermanT5-RP-Mod/                          ray_results/
RP-Mod/                                   results/
RP-Mod-GermanT5-oscar-german-small-el32/  t5_base_imdb_sentiment/
T5FineTuner.py                            t5_german_small_rp_mod/
aclImdb/                                  t5_german_small_rp_mod_2/
aclImdb_v1.tar.gz                         t5_german_small_rp_mod_3/
arguments_test_dir/                       t5_imdb_sentiment/
create_t5_embeddings.ipynb                wandb/
lightning_logs/


In [64]:
from T5FineTuner import T5FineTuner, RPDataset
from utils import get_folds
import torch
import argparse
from transformers import T5Tokenizer
from torch.utils.data import Dataset, DataLoader

DATASET = "RP-Crowd-3"
MODEL_NAME_OR_PATH = "GermanT5/t5-efficient-oscar-german-small-el32"
WANDB_PROJECT_NAME = "rp-crowd-3-folds-t5-efficient-small-el32"
OUTPUT_DIR = "./GermanT5-RP-Mod/t5-efficient-oscar-german-small-el32/"
SOURCE = f"./Datasets/{DATASET}-folds.csv"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME_OR_PATH)
checkpoint_path = "./GermanT5-RP-Mod/t5-efficient-oscar-german-small-el32/lr-0.0004-wd-0.1/epoch=3-val_accuracy=0.74-val_loss=0.28.ckpt"
train_inputs, train_targets, val_inputs, val_targets = get_folds(SOURCE)

train_dataset = RPDataset(tokenizer, train_inputs, train_targets)
valid_dataset = RPDataset(tokenizer, val_inputs, val_targets)

checkpoint = torch.load(checkpoint_path)
hparams = checkpoint["hyper_parameters"]
hparams["train_dataset"] = train_dataset
hparams["val_dataset"] = valid_dataset
new_args = argparse.Namespace(**hparams)

new_model = T5FineTuner.load_from_checkpoint(checkpoint_path, hparams=new_args)


### Load checkpoint of Encoder Only Model

In [17]:
from classification_classes import Enc1T5, RPClassificationDataset, load_dataset
import torch
import argparse
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader

DATASET = "RP-Crowd-2"
MODEL_NAME_OR_PATH = "GermanT5/t5-efficient-oscar-german-small-el32"
source = f"./Datasets/{DATASET}-folds.csv"

# tokenizer = AutoTokenizer.from_pretrained()
# train_dataset, valid_dataset = load_dataset(source)

checkpoint_path = "/home/dobby/bestmodels/enc1t5/epoch=3-val_accuracy=0.85.ckpt"                 
checkpoint = torch.load(checkpoint_path)
hparams = checkpoint["hyper_parameters"]
# hparams["train_dataset"] = train_dataset
# hparams["val_dataset"] = valid_dataset
new_args = argparse.Namespace(**hparams)

# now I am confused... what do I do here??
model = Enc1T5.load_from_checkpoint(checkpoint_path, hparams=new_args)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Some weights of the model checkpoint at GermanT5/t5-efficient-oscar-german-small-el32 were not used when initializing T5EncoderModel: ['decoder.block.1.layer.2.layer_norm.weight', 'decoder.block.1.layer.2.DenseReluDense.wi.weight', 'decoder.block.3.layer.0.layer_norm.weight', 'decoder.block.3.layer.0.SelfAttention.k.weight', 'decoder.block.3.layer.2.DenseReluDense.wo.weight', 'decoder.block.3.layer.2.DenseReluDense.wi.weight', 'decoder.block.2.layer.1.EncDecAttention.k.weight', 'decoder.block.4.layer.1.layer_norm.weight', 'decoder.embed_tokens.weight', 'decoder.block.2.layer.1.layer_norm.weight', 'decoder.block.5.layer.2.layer_norm.weight', 'decoder.block.5.layer.1.EncDecAttention.o.weight', 'decoder.block.4.layer.2.DenseReluDense.wi.weight', 'decoder.block.1.la

### Get False Positives

In [ ]:
# apply the model to the entire validation dataset


In [18]:
import numpy as np
import scipy as sp
import shap

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)

# define a score function
def f(x):
    """A function which takes a list of examples, x and returns a list of scores, val"""
    # print(len(x))
    # print(x)
    batch = tokenizer.batch_encode_plus(
        list(x), max_length=512, padding=True, truncation=True, return_tensors="pt"
    )
    outputs = model(batch["input_ids"], batch["attention_mask"])["prediction"].detach().numpy()
    # print(tv)
    # print(outputs)
    # apply softmax to each row??
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    # print(scores)
    val = scores[:,1] # use one vs rest logit units
    # print(val)
    # val needs to be a list of the score
    return val

false_pos = np.load("./false_pos/RP-Mod-false-pos.csv.npy", mmap_mode="r")
false_pos = [s[15:] for s in false_pos]
# f(list(false_pos[:10]))

explainer = shap.Explainer(f, tokenizer)


# shap_values = explainer(false_pos[:10], batch_size=8)
shap_values = explainer(false_pos, fixed_context=1, batch_size=8)

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer:   4%|▍         | 6/139 [00:14<03:14,  1.46s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer:   5%|▌         | 7/139 [00:22<08:22,  3.81s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

...

Partition explainer:  96%|█████████▋| 134/139 [12:28<00:39,  7.93s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer:  97%|█████████▋| 135/139 [12:43<00:40, 10.08s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 140it [13:02,  5.75s/it]                         


In [19]:
import heapq
words = {}

for val in shap_values:
    for i in range(len(val.values)):
        score = val.values[i]
        if score >= 0:
            word = val.data[i]
            if word not in words.keys():
                words[word] = [score]
            else:
                words[word].append(score)
for key in words.keys():
    words[key] = np.mean(words[key])
words
heapq.nlargest(200, words, key=words.get)

[' Migranten',
 ' Nazi',
 'arme',
 'Muslim',
 'monster   ',
 'ackt',
 ' Psychiater',
 'sy',
 ' Juden',
 'gewalt',
 'flüchtling',
 ' Männer',
 ' Muslime',
 'M',
 'täter',
 ' Typen',
 ' Mann',
 ' Öko',
 'äh',
 ' Radikal',
 'heit',
 ' Virus',
 'geschützt ',
 ' Kur',
 'Ideologie',
 ' Lebewesen',
 'gewandert',
 ' Terrorismus',
 ' DE',
 ' Mund',
 'ler',
 ' islamische',
 'gesperrt',
 'ten',
 ' Schweine',
 ' In',
 'staaten',
 'brecher',
 'fisch',
 ' Asyl',
 ' Rechts',
 ' Rechten',
 'hirn',
 ' Frau',
 'DW',
 ' Demo',
 ' fremden',
 ' Türkei',
 ' Youtube',
 ' Auge',
 'Hö',
 'ierer',
 ' hilflos',
 'deutsche',
 ' Flüchtlings',
 ' Oma',
 'eten',
 'Krieg',
 'din',
 ' Opfer',
 ' Familie ',
 ' dir ',
 ' intelligente',
 ' Falsch',
 'cke',
 'stern',
 ' Putin',
 'Na',
 'feindlichkeit ',
 ' Islam',
 ' Maul',
 'den',
 'lein',
 ' christlichen',
 ' dieses',
 ' Anti',
 ' Einwanderung',
 'NE',
 'flotte',
 ' deutsche',
 'frei',
 'tuch',
 'EI',
 ' seit',
 ' Täter',
 ' homo',
 ' Vornamen',
 'Veteran',
 ' Sozial',


In [71]:
shap.plots.text(shap_values[0])

In [32]:
#TODO: remove "Kurden" (or other words which SHAP identifies as import) 
# from the comment and then see what score the model gives it :) 
import numpy as np

txt = "Ich stelle mal folgende Vermutung an so wie es auch bei Tätern hier immer dargestellt wird Einzeltäter verwirrt traumatisiert. Liebe über alles"
def score_text(txt):

    input_ids = torch.tensor([tokenizer.encode(txt)])
    pred = model(input_ids)["prediction"].detach().numpy()
    # print(pred)
    scores = (np.exp(pred).T / np.exp(pred).sum(-1)).T
    return scores
score_text(txt)

array([[0.50282335, 0.49717662]], dtype=float32)

In [54]:
score_text("Ich liebe die Umwelt und mein Leben ist sehr schoen Diese Artikel wurde sehr gut geschrieben danke Mann" + "Liebe alles" * 100)

array([[0.5951737 , 0.40482628]], dtype=float32)

In [ ]:
score_text("ich liebe dich")

In [58]:
score_text("Ich liebe Katzen "*100)
# hello??? this can't be right :(

array([[0.6666064, 0.3333936]], dtype=float32)

In [59]:
true_negatives = np.load("./true_neg/RP-Mod-true-neg.npy")
sum([score_text(true_negatives[i])[0][1] >= 0.5 for i in range(len(true_negatives))])/len(true_negatives)
# score_text(true_negatives[6])

0.3873578

In [63]:
true_negatives[7]

'classification: Weil das Land damit einen einstelligen Milliardenbetrag an Subventionen spart '

In [24]:
f(false_pos[1:2])

array([0.22180541], dtype=float32)

In [23]:
false_pos[:10]

memmap(['classification Sehr sympathisch diese Frau und das sagt hier ein  Mann ',
        'classification Kein Artikel wert',
        'classification Na ja Wenigstens hat sie diesmal nicht Deutschland verbrüht Schonmal eine Entwicklungsstufe weiter',
        'classification Hallo nehmt doch die Rocker aus Erkrath Unterfeldhaus Die Jungens sind froh über eine solche Arbeit Wer sich nicht benimmt der fliegt und zwar raus',
        'classification Wildtiere haben im Zirkus nichts verloren   Inkompetente haben in der Politik nichts zu suchen ',
        'classification Ich stelle mal folgende Vermutung an so wie es auch bei Tätern mit Migrationshintergrund hier immer dargestellt wirdEinzeltäter verwirrt traumatisiert hat nix mit den Rechtsextremen zu tun',
        'classification Ich freue mich zu wissen das Deutschland in Brüssel von einer vertreten wird die Islamwissenschaft und Turkologie studiert hat  Sehr hilfreich für die vielen neuen Gäste Ska wird ganz ganz viel Verständnis für die

In [15]:
false_pos[:10]

memmap(['classification Sehr sympathisch diese Frau und das sagt hier ein  Mann ',
        'classification Kein Artikel wert',
        'classification Na ja Wenigstens hat sie diesmal nicht Deutschland verbrüht Schonmal eine Entwicklungsstufe weiter',
        'classification Hallo nehmt doch die Rocker aus Erkrath Unterfeldhaus Die Jungens sind froh über eine solche Arbeit Wer sich nicht benimmt der fliegt und zwar raus',
        'classification Wildtiere haben im Zirkus nichts verloren   Inkompetente haben in der Politik nichts zu suchen ',
        'classification Ich stelle mal folgende Vermutung an so wie es auch bei Tätern mit Migrationshintergrund hier immer dargestellt wirdEinzeltäter verwirrt traumatisiert hat nix mit den Rechtsextremen zu tun',
        'classification Ich freue mich zu wissen das Deutschland in Brüssel von einer vertreten wird die Islamwissenschaft und Turkologie studiert hat  Sehr hilfreich für die vielen neuen Gäste Ska wird ganz ganz viel Verständnis für die

In [25]:
from sklearn import metrics
val_dataloader = DataLoader(valid_dataset, batch_size=64)

# for batch in val_dataloader:
batch = next(iter(val_dataloader))
labels = batch["target_ids"]
outs = new_model.model.generate(input_ids=batch["source_ids"], 
                            attention_mask=batch["source_mask"], 
                            max_length=2)

dec = [tokenizer.decode(ids) for ids in outs]
target = [tokenizer.decode(label) for label in labels]

new_outputs = [s[6:] for s in dec]
new_targets = [s[:-4] for s in target]

f1 = metrics.f1_score(new_targets, new_outputs, pos_label="problematisch")
print(f1)

0.7419354838709677
